**Set environment**

In [1]:
suppressMessages(suppressWarnings(source("../config/config_sing.R")))
show_env()

You are in Singularity: singularity_proj_encode_fcc 
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei 
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/out 
CODE DIRECTORY (FD_CODE): /data/reddylab/Kuei/code 
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC 
PATH OF RESULTS (FD_RES): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc 
PATH OF LOG     (FD_LOG): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log 


**Check data**

In [2]:
fdiry = file.path(
    FD_RES, 
    "results", 
    "region", 
    "KS91_K562_ASTARRseq_peak_macs_input", 
    "annotation_chipseq_tf")
fnames = dir(fdiry)
print(length(fnames))
print(head(fnames))

[1] 733
[1] "peak.annotation.K562.ENCSR000AKO.ENCFF769AUF.CTCF.bed.gz" 
[2] "peak.annotation.K562.ENCSR000AQA.ENCFF807FNB.KDM5B.bed.gz"
[3] "peak.annotation.K562.ENCSR000AQD.ENCFF408NUX.CHD1.bed.gz" 
[4] "peak.annotation.K562.ENCSR000AQE.ENCFF804RVA.EZH2.bed.gz" 
[5] "peak.annotation.K562.ENCSR000AQF.ENCFF669MJX.HDAC1.bed.gz"
[6] "peak.annotation.K562.ENCSR000AQG.ENCFF182MPT.HDAC2.bed.gz"


In [4]:
lst = str_split(string = fnames, pattern = "\\.")
for (vec in head(lst)){
    index  = vec[5]
    target = vec[6]
    label  = paste(target, index, sep="_")
    cat(vec,   "\n")
    cat(label, "\n")
    cat("\n")
}

peak annotation K562 ENCSR000AKO ENCFF769AUF CTCF bed gz 
CTCF_ENCFF769AUF 

peak annotation K562 ENCSR000AQA ENCFF807FNB KDM5B bed gz 
KDM5B_ENCFF807FNB 

peak annotation K562 ENCSR000AQD ENCFF408NUX CHD1 bed gz 
CHD1_ENCFF408NUX 

peak annotation K562 ENCSR000AQE ENCFF804RVA EZH2 bed gz 
EZH2_ENCFF804RVA 

peak annotation K562 ENCSR000AQF ENCFF669MJX HDAC1 bed gz 
HDAC1_ENCFF669MJX 

peak annotation K562 ENCSR000AQG ENCFF182MPT HDAC2 bed gz 
HDAC2_ENCFF182MPT 



## Import data

In [5]:
fdiry = file.path(
    FD_RES, 
    "results", 
    "region", 
    "annotation_chipseq_tf")
fname = "description.tsv"
fpath = file.path(fdiry, fname)

dat = read_tsv(fpath)

dat_cnames = dat
print(dim(dat))
dat

Rows: 10 Columns: 2
── Column specification ──────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr (2): Name, Description

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] 10  2


Name,Description
<chr>,<chr>
Chrom,Chromosome
Start,Start position
End,End position
Name,Name given to a region
Score,"Scaled IDR value, min(int(log2(-125IDR), 1000)."
Strand,[+-.]; Use '.' if no strand is assigned.
SignalValue,Measurement of enrichment for the region for merged peaks.
PValue,Measurement of statistical significance (-log10). Use -1 if no pValue is assigned.
QValue,Measurement of statistical significance using false discovery rate (-log10). Use -1 if no qValue is assigned.


In [6]:
annotation = "ChIPseq_TF"
cnames = dat_cnames$Name
cnames = c("Chrom_ATAC", "Start_ATAC", "End_ATAC", cnames, "Overlap")

fdiry = file.path(
    FD_RES, 
    "results", 
    "region", 
    "KS91_K562_ASTARRseq_peak_macs_input", 
    "annotation_chipseq_tf")

lst = str_split(string = fnames, pattern = "\\.")
lst = lapply(lst, function(vec){
    
    ### parse info
    index  = vec[5]
    target = vec[6]
    label  = paste(target, index, sep="_")
    
    ### show progress
    cat("Read:", label, "\n"); flush.console()
    
    ### set file path
    fname = paste("*", index, target, "bed.gz", sep = ".")
    fglob = file.path(fdiry, fname)
    fpath = Sys.glob(fglob)
    
    ### read data
    dat = read_tsv(fpath, col_names = cnames, show_col_types = FALSE)
    dat = dat %>% dplyr::mutate(
        Peak_ATAC  = paste(Chrom_ATAC, Start_ATAC, End_ATAC, sep="_"),
        #Peak_ATAC  = paste0(Chrom_ATAC, ":", Start_ATAC, "_", End_ATAC),
        Annotation = annotation,
        Label      = label
    )
    return(dat)
})

lst_peak_annot = lst
print(length(lst))
head(lst[[1]])

Read: CTCF_ENCFF769AUF 
Read: KDM5B_ENCFF807FNB 
Read: CHD1_ENCFF408NUX 
Read: EZH2_ENCFF804RVA 
Read: HDAC1_ENCFF669MJX 
Read: HDAC2_ENCFF182MPT 
Read: PHF8_ENCFF981ISM 
Read: RBBP5_ENCFF942VGF 
Read: SAP30_ENCFF383IEP 
Read: HDAC6_ENCFF736FGN 
Read: CHD4_ENCFF985QBS 
Read: REST_ENCFF539MIO 
Read: CREBBP_ENCFF532VPN 
Read: CBX2_ENCFF258XBJ 
Read: CBX3_ENCFF068OEJ 
Read: CBX8_ENCFF522HZT 
Read: KDM1A_ENCFF054XCG 
Read: NCOR1_ENCFF816AEF 
Read: KAT2B_ENCFF349VSP 
Read: RNF2_ENCFF923GDR 
Read: SIRT6_ENCFF821XJU 
Read: SUZ12_ENCFF647TBC 
Read: SETDB1_ENCFF267QRI 
Read: CHD7_ENCFF722UJW 
Read: WHSC1_ENCFF862UUR 
Read: SPI1_ENCFF888CKG 
Read: SIX5_ENCFF742ZHT 
Read: ZBTB33_ENCFF146GZZ 
Read: BCLAF1_ENCFF070ZTX 
Read: GATA2_ENCFF242YZU 
Read: SP1_ENCFF171NEU 
Read: ETS1_ENCFF886BDQ 
Read: POLR2AphosphoS5_ENCFF060MMW 
Read: TAF1_ENCFF784BTI 
Read: USF1_ENCFF310CCS 
Read: YY1_ENCFF074OAM 
Read: RAD21_ENCFF930WPG 
Read: E2F6_ENCFF831NDQ 
Read: SRF_ENCFF101SEZ 
Read: GABPA_ENCFF489EME 
Read: MAX

Chrom_ATAC,Start_ATAC,End_ATAC,Chrom,Start,End,Name,Score,Strand,SignalValue,PValue,QValue,Peak,Overlap,Peak_ATAC,Annotation,Label
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>
chr1,267853,268603,chr1,267966,268092,.,1000,.,36.26682,-1,4.33816,58,126,chr1_267853_268603,ChIPseq_TF,CTCF_ENCFF769AUF
chr1,777949,779437,chr1,778701,778979,.,1000,.,78.32446,-1,4.65032,175,278,chr1_777949_779437,ChIPseq_TF,CTCF_ENCFF769AUF
chr1,777949,779437,chr1,778870,779294,.,1000,.,22.45062,-1,4.08853,212,424,chr1_777949_779437,ChIPseq_TF,CTCF_ENCFF769AUF
chr1,869623,870250,chr1,869772,870056,.,1000,.,140.67100,-1,4.65032,149,284,chr1_869623_870250,ChIPseq_TF,CTCF_ENCFF769AUF
chr1,903633,905556,chr1,904625,904954,.,1000,.,150.99362,-1,4.65032,154,329,chr1_903633_905556,ChIPseq_TF,CTCF_ENCFF769AUF
chr1,912650,914455,chr1,912808,913232,.,1000,.,49.29953,-1,4.25610,212,424,chr1_912650_914455,ChIPseq_TF,CTCF_ENCFF769AUF


## Arrange and summarize

In [7]:
lst = lst_peak_annot

lst = lapply(lst, function(dat){
    dat = dat %>% 
        dplyr::group_by(Peak_ATAC, Annotation, Label) %>%
        dplyr::summarise(Count = n(), .groups = "drop")
    return(dat)
})

dat = bind_rows(lst)
dat = dat %>% dplyr::rename("Peak" = "Peak_ATAC")

dat_peak_annot_count = dat
print(dim(dat))
head(dat)

[1] 5716412       4


Peak,Annotation,Label,Count
<chr>,<chr>,<chr>,<int>
chr10_100009096_100010466,ChIPseq_TF,CTCF_ENCFF769AUF,1
chr10_100045961_100046653,ChIPseq_TF,CTCF_ENCFF769AUF,1
chr10_100228452_100230090,ChIPseq_TF,CTCF_ENCFF769AUF,1
chr10_100240591_100241246,ChIPseq_TF,CTCF_ENCFF769AUF,1
chr10_100338359_100339077,ChIPseq_TF,CTCF_ENCFF769AUF,1
chr10_100342247_100342824,ChIPseq_TF,CTCF_ENCFF769AUF,1


In [8]:
fdiry = file.path(FD_RES, "results", "region", "KS91_K562_ASTARRseq_peak_macs_input", "summary")
fname = "peak.summary.chipseq_tf.tsv"
fpath = file.path(fdiry, fname)

dat = dat_peak_annot_count
write_tsv(dat, fpath)